In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from aqara_viewer import create_aqara_viewer

In [3]:
# AQARAPATH = "/mnt/cameras/aqara_video"
AQARAPATH = "/mnt/hdd/diegocaro/aqara_video"
create_aqara_viewer(AQARAPATH)